In [3]:
import cv2
import numpy as np
import pickle
from matplotlib import pyplot as plt

In [7]:
def skin_detection(img):
    
    skinLow = np.array([0,48,80],np.uint8)
    skinHigh = np.array([20,255,255],np.uint8)
    img = cv2.GaussianBlur(img,(5,5),0)
    imgHSV = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(imgHSV,skinLow,skinHigh)
    result = cv2.bitwise_and(img,img,mask=mask)
    imgGray = cv2.cvtColor(result,cv2.COLOR_BGR2GRAY)
    
    return imgGray

In [8]:
winSize = (128,128)
blockSize = (16,16)
blockStride = (4,4)
cellSize = (8,8)
nbins = 9
derivAperture = 1
winSigma = -1.
histogramNormType = 0
L2HysThreshold = 0.2
gammaCorrection = 1
nlevels = 64

hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,
                        derivAperture,winSigma,histogramNormType,
                        L2HysThreshold,gammaCorrection,nlevels)

In [13]:
img = pickle.load(open("img_save.p","rb"))

In [15]:
pre = []
hog_data = []
hog_data_flat = []

for n in img:
    pre.append(skin_detection(n))
for n in pre:
    hog_data.append(hog.compute(n))
for n in hog_data:
    hog_data_flat.append(n.flatten())

In [16]:
hog_data_flat[0]

array([0.        , 0.        , 0.        , ..., 0.4300235 , 0.35288692,
       0.10567046], dtype=float32)

In [23]:
x = np.float32(hog_data_flat).reshape(-1,30276)
y = np.float32(np.repeat(np.arange(24),19)[:,np.newaxis])

In [31]:
from sklearn.model_selection import cross_val_score
from sklearn import svm

clf = svm.SVC()
cross_val = cross_val_score(clf, x, y.ravel(), cv=10)
cross_val

array([0.93478261, 0.76086957, 0.58695652, 0.65217391, 0.76086957,
       0.84782609, 0.84444444, 0.91111111, 0.84444444, 0.8       ])

In [32]:
print(np.mean(cross_val))

0.7943478260869565


In [52]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

scores = []
clf_report = []
conf_x = []
y_numbers = []

clf_svm = svm.SVC()
cv = KFold(n_splits=10,shuffle=True)

for train_index, test_index in cv.split(x):

    X_train, X_test = x[train_index], x[test_index] 
    Y_train, Y_test = y[train_index].ravel(), y[test_index].ravel()
    
    clf_svm.fit(X_train, Y_train)
    pred = clf_svm.predict(X_test)
    
    y_numbers.append(Y_test)
    
    conf_x.append(confusion_matrix(Y_test, pred))
    clf_report.append(classification_report(Y_test,pred))
    scores.append(clf_svm.score(X_test, Y_test))

/home/not/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/not/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/not/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/not/.local/lib/python3.6/site-packages/sklearn/metr

In [53]:
print(scores)
print('\n')
print(np.mean(scores))

[0.6956521739130435, 0.6956521739130435, 0.7608695652173914, 0.6521739130434783, 0.7608695652173914, 0.7608695652173914, 0.8444444444444444, 0.7777777777777778, 0.9555555555555556, 0.7777777777777778]


0.7681642512077296


In [54]:
print(clf_report[7])

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00         3
         1.0       0.75      1.00      0.86         3
         2.0       0.67      0.67      0.67         3
         3.0       0.00      0.00      0.00         1
         4.0       1.00      1.00      1.00         3
         5.0       1.00      0.67      0.80         3
         6.0       1.00      1.00      1.00         2
         7.0       1.00      1.00      1.00         3
         8.0       1.00      0.67      0.80         3
         9.0       1.00      1.00      1.00         1
        10.0       1.00      1.00      1.00         2
        11.0       0.00      0.00      0.00         3
        12.0       0.25      1.00      0.40         1
        13.0       0.67      1.00      0.80         2
        14.0       0.00      0.00      0.00         0
        15.0       0.00      0.00      0.00         2
        16.0       0.00      0.00      0.00         0
        17.0       1.00    